In [1]:
from time import time
import string
import random
from pprint import pprint

# Algorytm naiwny 
### złożoność $O(m \cdot(n-m+1))$

In [2]:
def naive_string_matching(text, pattern, printt = True):
    t = time()
    for s in range(0, len(text) - len(pattern) + 1):
        if pattern == text[s:s + len(pattern)] and printt:
            print(f"Przesunięcie {s} jest poprawne")
    return time() - t

# Algorytm automatu skończonego 

### złożoność dopasowania $O(n)$

In [3]:
def fa_string_matching(text, pattern, printt = True, res = []):
    tt = time()
    q = 0
    delta = transition_table(pattern)

    t = time()
    
    for s in range(len(text)):
        q = delta[q].get(text[s], 0) # jesli nie ma znaku w tablicy przejscia 
                                        # to domyslnie wrzuca 0

        if (q == len(delta) - 1):
            if printt:
                print(f"Przesunięcie {s + 1 - q} jest poprawne")
            # s + 1 - ponieważ przeczytaliśmy znak o indeksie s, 
                # więc przesunięcie jest po tym znaku
            res.append(s+1-q)
    return time() - tt, time() - t 

### złożoność obliczenia tablicy przejścia $ O(m^3 \cdot |\sum|) $, gdzie $|\sum|$ oznacza wielkość alfabetu

In [4]:
def check_pattern(pattern, text):
    return pattern == text[(len(text) - len(pattern)):]

def transition_table(pattern):
    result = []

    alphabet = set(pattern)

    for q in range(len(pattern) + 1):
        result.append({})
        for a in alphabet:
            k = min(len(pattern), q + 1)
            while not check_pattern(pattern[:k], pattern[:q] + a):            
                k = k - 1

            result[q][a] = k
    return result

In [5]:
pattern = "ABAC"
s = time()
pprint(transition_table(pattern))
print("time:",time() - s)

[{'A': 1, 'B': 0, 'C': 0},
 {'A': 1, 'B': 2, 'C': 0},
 {'A': 3, 'B': 0, 'C': 0},
 {'A': 1, 'B': 2, 'C': 4},
 {'A': 1, 'B': 0, 'C': 0}]
time: 0.0014908313751220703


# Algorytm kmp (Knutha-Morrisa-Pratta)

### złożoność dopasowania $O(n)$

In [6]:
def kmp_string_matching(text, pattern, printt = True, res = []):
    tt = time()
    pi = prefix_function(pattern)
    t = time()
    q = 0
    
    for i in range(len(text)):
        while q != 0 and pattern[q] != text[i]:
            q = pi[q - 1]

        if pattern[q] == text[i]:
            q = q + 1
        if q == len(pattern):
            if printt:
                print(f"Przesunięcie {i + 1 - q} jest poprawne")
            res.append(i+1-q)
            q = pi[q - 1]
    return time() - tt, time() - t

### złożoność obliczania funkcji przejścia $O(m)$

In [7]:
def prefix_function(pattern):
    pi = [0] * len(pattern)
    k = 0
    for q in range(1, len(pattern)):
        while k != 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]

        if pattern[k] == pattern[q]:
            k = k + 1
        pi[q] = k
    return pi

# Porównanie algorytmów

In [8]:
def speed_tests(text, pattern):
    naive = naive_string_matching(text, pattern, False)
    print("naive:        ", naive)
    print()
    print("fa")
    fa_total, fa_matching = fa_string_matching(text, pattern, False)
    print("total:        ", fa_total)
    print("matching:     ", fa_matching)
    print("preprocessing:", fa_total - fa_matching)
    fa_pre = fa_total - fa_matching
    
    print()
    print("kmp")
    kmp_total, kmp_matching = kmp_string_matching(text, pattern, False)
    print("total:        ", kmp_total)
    print("matching:     ", kmp_matching)
    print("preprocessing:", kmp_total - kmp_matching)
    kmp_pre = kmp_total - kmp_matching
    
    print()
    print("naive / fa matching: ", 
          (naive / fa_matching) if fa_matching != 0 else float("inf"))
    print("naive / kmp matching:", 
          (naive / kmp_matching) if kmp_matching != 0 else float("inf"))
    print()
    print("fa preprocessing / kmp preprocessing:", 
          (fa_pre/kmp_pre) if kmp_pre != 0 else float("inf"))

## Przykładowy tekst ustawy, w którym szukamy wzorca "art"

In [9]:
text = open("ustawa", encoding="utf8").read()
pattern = "art"

## Porównanie prędkości algorytmów dla ustawy

In [10]:
speed_tests(text, pattern)

naive:         0.037450313568115234

fa
total:         0.0326838493347168
matching:      0.03266000747680664
preprocessing: 2.384185791015625e-05

kmp
total:         0.02471637725830078
matching:      0.024712324142456055
preprocessing: 4.0531158447265625e-06

naive / fa matching:  1.1466719226782298
naive / kmp matching: 1.515450888076333

fa preprocessing / kmp preprocessing: 5.882352941176471


### Przypadek, gdy prędkości dopasowania algorytmów liniowych są przynajmniej 5 razy krótsze niż dla algorytmu naiwnego

In [11]:
text = "a"*1000000
pattern = "a"*50000
speed_tests(text, pattern)

naive:         1.7829809188842773

fa
total:         0.5489776134490967
matching:      0.37734246253967285
preprocessing: 0.17163515090942383

kmp
total:         0.37167787551879883
matching:      0.3649747371673584
preprocessing: 0.00670313835144043

naive / fa matching:  4.725100130221414
naive / kmp matching: 4.885217351541499

fa preprocessing / kmp preprocessing: 25.605192957495998


### Skrajnie niekorzystny przypadek dla algorytmu automatu skończonego, czas preprocessingu jest skrajnie długi.

In [12]:
text = ''.join(random.choice("0123456789") for i in range(100000))
pattern = ''.join([str(i) for i in range(500)])

print("dlugosc wzorca:",len(pattern),"\n")
speed_tests(text, pattern)

dlugosc wzorca: 1390 

naive:         0.021320819854736328

fa
total:         3.577767848968506
matching:      0.014858484268188477
preprocessing: 3.5629093647003174

kmp
total:         0.014928579330444336
matching:      0.01482701301574707
preprocessing: 0.00010156631469726562

naive / fa matching:  1.4349256269957158
naive / kmp matching: 1.437971345414784

fa preprocessing / kmp preprocessing: 35079.63615023474


### Przypadek gdy algorytm naiwny jest najszybszy

In [13]:
text = "a" *10000000
pattern = "aaaaaaaabcdeeeeaaaaaaaaaafg"*10
speed_tests(text, pattern)

naive:         1.4546284675598145

fa
total:         1.7159547805786133
matching:      1.6516203880310059
preprocessing: 0.06433439254760742

kmp
total:         2.290165662765503
matching:      2.290133237838745
preprocessing: 3.24249267578125e-05

naive / fa matching:  0.8807280886705224
naive / kmp matching: 0.6351719819291314

fa preprocessing / kmp preprocessing: 1984.1029411764705
